### Abstract


Through structured experimentation this assignment explores and demonstrates a number of roperties of artificial Neural networks. Using the CIFAR-10 dataset and classification task and a relatively simple convolutional neural network, the effect that a number of network and paramaeter choices shall be explored. 

Looking first at varying learning rates and learning rate decay then at dropout for regularisation, transfer learning and batch normalisation, it is hoped that a number of important and essential properties shall be described and demonstrated. 

Whilst the aim of the task is not strictly to aim for best peformance, the impacy that the different tecnicques and choices make on performance shall be discussed. 

It shall be shown that.........

training dynamics and performance of    

### Introduction

The labelled CIFAR-10 dataset was created as part of study exploring different approaches to training generative models for natural images [3]. It and it's larger sibling CIFAR-100 have been used for benchmarking and testing in many exploratory and ground breaking papers relating to computer vision and image classification since, not least in the development of Alexnet [x],  Resnet [y] and most recently in the development of  transformer-for vision architectures [z], as well as many others [a, bm c]. It is fitting, then, to use it to explore some of the fundamental properties of aritificial neural networks (NN) in this assignment.

The objective of this assignment examine the impact of NN design choices on performance and gradient flow in a deep convulutional neural network trained to perform multiclass classification on CIFAR images. This shall be done through 3 experiments, each examining a different property of NN model learning. 

The first experiment examined the effect that altering the learning rate has on training and performance. As well as experimenting with different learning rates, a learning rate 'scheduler' was designed and its performance compared to a high performing learning unsscheduled learning. 

The second experiment aimed to understand and demonstrate the impact of introducing a dropout layer into the arhchitecture of the network. Different dropout rates were trialled and their effects compared to baseline performance in both training and evaluation. The effect of dropout was also tested in a transfer learning context.

The third experiment was focused on understanding the effects of both dropout and batch normalisation on gradient flow in the network during backpropagation. It was designed to show the impact that these interventions have on the propogation of the gradient through the different layers of the network at different points of the training cycle. 

The experiments showed.....

## Data (7 MARKS) <ignore>

As already mentioned, the data used in this assignment is the CIFAR-10 dataset developed by Krizhevsky and Hinton, et al. [x]. It consists of 60,000 colour 32x32 images, each of which belongs to one 10 mutually exclusive classes and is labelled correspondingly. These classes describe the suject of the image and are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck.

The Cifar-10 dataset, along with many others, is available for download via a conveient Pytorch `datasets` method which uses a boolean flag to enable the user to load both the 50,000 training images and 10,000 test images into separate `torch.Dataset` instances extremely easily, and this was the method used here. As part of this process it is possible to apply manual transforms to the data as it is loaded. Using the `Normalize` method the data so that the pixel values of each of the 3 input channels for all of the datasets has a mean of 0 and and a standard deviation of 1, the idea being to focus the learning on the underlying properties of the data rather than any incidental variance in the raw data.  

The training set consisting of 50,000 labelled samples was then split to create a validation or development set of 5000 samples (with a random seed set for consistence). The final numbers for training, validation and testing then were 45,000, 5,000 and 10,000 respectively. The class distribution for for each dataset were checked and found to be well balanced which means that accuracy should be a reliable measure of overall performance of the model on test data. (althgouh this differed in experiments 2 and 3)

Batching for stochastic gradient descent was handled by the `DataLoader` class, which yields samples from the shuffled dataset without replacement in batches of a size that can be specified by the user until the data has been exhauted. The data is then shuffled again and new batches are drawn from the newly shuffled data so that the batches in every epoch were different, encouraging more stoacasticity - a key element in SGD being said to approximat the true gradient of the loss. 

It was decided that a simple train, validation, test split would be appropriate for the task at hand. Cross-validation is good for getting a very accurate idea of the likely performance of a model on the test set, and exposes the model to all of the possible training data. However, performance was not the chief aim of this assignment - rather it was understanding and the comparison of performance on training, validation and a test set is sifficient.   


<figure><center><img src="./figs/classdisttraining.png" width=300><img src="./figs/classdistval.png" width=300><img src="./figs/class dist test.png" width=300><figcaption style="max-width: 600px"> Figure 1. Dropout is more effective in convolutional layers when applied to whole activation maps</figcaption></center></figure>


## Architecture (17 MARKS) <ignore>


<figure><center><img src="./figs/baseline_model_diagram.png" width=800><figcaption style="max-width: 600px"> Figure 1. BaselineNet Convolutional Neural Network. </figcaption></center></figure>


The baseline architecture was designed in accordance with the assigonment brief and some intitial experimentation with hyperparamaters.

The basic architecture visable in Fig 1. shall be outlined in more detail and then reasoning and results that led to some of the decisions shall be discussed.

The inputs to the model are the 32 x 32 colour images which have a depth of 3 owing to the RGB channels. These are convolved with the first convolutional layer `conv1` comprised of 16 filters as outlined in `Table 1` below above. The resultant output was passed through a relu activation and put through a max pooling filter `pool1` which compacts the previous layers spatial dimensions by half whilst preserving the number of channels. It should be noted pooling layers `pool2` and `pool3` perform the same operation and have the same structure. The output of `pool1` was convolved with `conv2` leading to a 32 channel, 16x16 output which were passed through a relu activation followed by another max pooling layer `pool2`. The final convolutional layers `conv3` increased the channel size to 64, before the activations were passed through a relu activation and then max pooled again before being `flattened` and passed to the first fully connected layer `fc1` and through aother relu activation into `fc2` and finally to the 10 dimensional output layer where it goes through a softmax actication.

<figure><center><img src="./figs/TABLE.PNG" width=600><figcaption style="max-width: 600px"> Table 1: Convolutional Neural Network Architecture</figcaption></center></figure>

The choice of hyperparameters for the BaselineNet architecture were made based on a combination of the assignment brief, initial experimentation, and common practices in the field.

The batch size of 64 was selected as a balance between computational efficiency and the ability to capture a representative sample of the dataset in each iteration. This size allows for efficient data processing on on GPU while providing a reasonable approximation of the gradient during training.

The filter dimensions of 3x3 were chosen as they have been shown to be effective in capturing local spatial patterns while keeping the number of parameters relatively low [VGG???]. 

The increasing number of filters (16, 32, 64) in the convolutional layers allows the network to learn progressively more complex and abstract features as the depth increases. 

The setting the stride combined with a padding of 1 in the convolutional layers ensures that the spatial resolution is preserved, while prevents information loss at the edges. 

The max pooling layers with a pool size of 2x2 and stride of 2 help to reduce the spatial dimensions, thereby reducing the number of parameters and providing a form of translation invariance. 

Overall, these choices strike a balance between model complexity, computational efficiency, and the ability to learn meaningful features from the CIFAR-10 dataset.

Owing to the number of training runs required to get accurate, averages results, parameter size was a legitimate consideration as it impacted on training time signfiriantly.  

```
The choice of ReLU (Rectified Linear Unit) as the activation function throughout the BaselineNet architecture is based on its proven effectiveness and computational efficiency. ReLU has become the default activation function for many deep learning models due to its ability to alleviate the vanishing gradient problem and promote sparse representations [ReLU_Advantages]. It introduces non-linearity into the network, allowing it to learn complex patterns and representations. ReLU is computationally efficient compared to other activation functions like sigmoid or tanh, as it involves a simple thresholding operation. This efficiency enables faster training and inference times. Additionally, ReLU has been shown to accelerate convergence during training by providing a consistent gradient flow [ReLU_Convergence].
Overall, these choices strike a balance between model complexity, computational efficiency, and the ability to learn meaningful features from the CIFAR-10 dataset. Owing to the number of training runs required to get accurate, averages results, parameter size was a legitimate consideration as it impacted on training time signfiriantly.
```


## Loss function (3 MARKS) <ignore>

The loss function used for each experiment was cross-entropy loss, implimented using the `nn.CrossEntropyLoss` class from Pytorch [x].

It is widely used method for quantifying loss function for classification problems such as this where the target is binomial or miultinomial. Cross-entropy works on logits that have been transformed by a softmax activation into what is a effectively a probability distribution across the output classes. It compares this output probability distribution to a one-hot encoded version of the class label, where the value at the index for the true class is 1, and all the others are 0. This acts as a target probability distribution and the cross entropy loss calculation function essentially quantifies the difference between this predicted distribution and the one-hot encoded true label distribution. 

Mathematically, for a single sample with true label <lt>$y$</lt> and predicted probabilities <lt>$\hat{y}$</lt>, the cross-entropy loss is calculated as:
<lt>$$\text{CE}(y, \hat{y}) = -\sum_{i=1}^{C} y_i \log(\hat{y}_i)$$</lt>

where <lt>$C$</lt> is the number of classes, <lt>$y_i$</lt> is the true label (0 or 1) for class <lt>$i$</lt>, and <lt>$\hat{y}_i$</lt> is the predicted probability for class <lt>$i$</lt>. By minimizing the average cross-entropy loss over all training samples, the model learns to assign high probabilities to the correct class and low probabilities to the incorrect ones.

The logarithm in the formula strongly penalizes misclassifications and encourages the model to produce well-calibrated probability estimates. Cross-entropy is a principled and effective loss function for multi-class classification, aligning with the maximum likelihood estimation objective. It drives the model to minmise the difference between its output probabilities and the true class probability, encouraging it to learn a mapping from features to the correct output probabilities. 

Practically, the use of the Pytorch module precludes the need for a softmax layer in the model architecture itself as the optimiser takes in the raw logits and then applys the `nn.LogSoftmax()` activation function [x] and the `nn.NLLLoss()` [x] (Negative Log-Likelihood Loss) in a single operation that encapsulates the above. The function used as it is here in a mini-batch stochastic gradient decent context also handles the averaging of the loss across the mini-batch. This averaging is important because it allows the loss to be invariant to the batch size and provides a stable estimate of the overall loss for the batch and then across batches in the epoch.

## Optimiser (4 MARKS) <ignore>

The optmiser used to handle parameter updates and impliment gradient descent was stochastic gradient descent (SGD), implimented using the `optim.SGD` class from Pytorch. 

SGD is the most straighforward and in many ways the original optimiser in artiificial neural networks. Aside from pure gradient descent calculated as an average of the gradients for the entire training dataset, it is as straightforward an approach to optimisation as one can use.

As an approach the idea is that it estimates the true gradient of the loss function using a small subset of the training data (a mini-batch) and update the parameters with this approximate gradient, weighted by a learning rate which, in this approach, is fixed, and user defined. 

This process is repeated for multiple mini-batch samples taken from the training data without replacement (until the entire data set has been seen - representing an 'epoch' of training) and then repeated until a stopping criterion is met - in this case a set number of epochs.

Mathematically, the estimated gradient for a mini-batch of size $B$ sampled from the training data is computed as:
<lt>$$\nabla_\theta L(\theta_t) \approx \frac{1}{B} \sum_{i=1}^{B} \nabla_\theta L(\theta_t; x_i, y_i)$$</lt>
where <lt>$(x_i, y_i)$</lt> represents the <lt>$i$</lt>-th example in the mini-batch.

There a number of more sophiticated optimisers availabl when training NNs today, not least the 'Adam' (Adaptive moment Estimation) optimiser [x] which is near uniquitous and recommended for most cases is "one of the more robust and effective optimization algorithms to use in deep learning" [x]. These approaches, by encoporating properties such as the 'momentum' of the gradient, as well as adaptive learning rates have been shown to allow for a smoother and more direct journey through paramater space to the miminimum loss. However, one of the objective was to explore the effect of the learning rate on performance, and with SGD the paramaters are directly updated the parameters based only on the gradient and the learning rate. By keeping to this very direct forumlation it easier to understand and interpret the impact of the learning rate on the model's performance - the learning rate has a clear and direct influence on the step size of the parameter updates, making it straightforward to study its effect. SGD is highly sensitive to the choice of the learning rate and this sensitivity is precisely what makes SGD suitable for studying the impact of learning rate on model performance. By varying the learning rate and observing the corresponding changes in model behavior, you can gain insights into the optimal learning rate range and its effect on convergence speed and generalization. The absence of adaptive learning rates in SGD in particular  ensures that the learning rate remains consistent throughout the training process.
This consistency allows for a clearer analysis of the relationship between the learning rate and model performance, without the confounding effects of adaptive learning rate schedules.

An interesting furthe development would be to introduce momentum and compare performance, then try Adagrad, then try Adam. But as the aim is just to explore learning rate, I felt it better to keep the optimiser algorithm as simple as possible. 


## Experiments <ignore>
### Experiment 1 (8 MARKS)

#### 1.1 - Learning Rates

Initial exploratory trials of single training runs for learning rates chosen between <lt>$0.5$</lt> and <lt>$1e^-6$</lt> established the extremes of model behaviour with respect to those learning rate (no learning at a learning rate of <lt>$1e^-3$</lt> or below, unstable learning at <lt>$0.15$</lt> or above. These invrestigation suggested a sensible range from which to select the 5 learning rates to compare which were <lt>$0.1, 0.075. 0,05, 0.025 \text{, and }0.01$</lt>

The data was loaded and split as described above. A batch size of 64 was used and the training run for 50 epochs for each trial. The inbuilt Pytroch SGD optimizer was used, along with the CrossEntropyLoss criterion.

For each learning rate, 5 trials were conducted. That is; 5 different models were instantiated, trained and evlauated for each learning rate in order to get a true sense of performance. The data for each trial was collated into a dictionary and stored JSON format so all data was accessable for visualisation. 

Individual trials were run as follows. A random seed was set before each model was initialised with the same 5 seeds used across all learning rates to ensure fair comparison as all models started with the same initial weights.  

Models were trained by mini-batch stoachastic gradient descent as desvined above. During training each batch was scored in terms of loss and accuracy.Batch scores were averaged across the epoch to give the training loss and accuracy for that epoch. After training for each epoch, the model was was then taken out of training mode - halting gradient computations - and the validation set was iterated through in batches, with the validation batch losses and accuracies again being averaged to give a validation loss and accuracy for the epoch. Smoothing was applied where validation volatitliy made results hard to interpred.

Accumulating these measures across epochs rather than batches is a somewhat aribitraty although conventional approach. It is a convenient way to keep track of how many times the model has been exposed to all of the training data and is easy to understand when plotting performance graphs. 

By running this experiment with different learning rates and multiple random seeds, a reasonable understanding of how the learning rate affects the model's performance can be obtained.

The averaged results provide insights into the model's training progress and generalization ability.

#### 1.2 - Learning Rate Scheduler

Having established above the performance of different learning rates it was clear that the model could tolerate a relatively high initial learning rate but this needed to drop significantly and arrive at or beneath 0.02 by the end of the 50 epochs to ensure a more fine grained exploration of the loss landscape in later stages. 

A number of approached to learning rate scheduling are available, but it made sense to try a number of simple appraoches first, so a simple step decay, exponential decay and inverse time decay were plotted and trialled to try and establish which ahcieved the desired shape of decay, and with what decay rate/schedule. This was found to be an inverse time function with a decay rate of 0.25  

<lt>$\alpha_t = \frac{\alpha_0}{1 + kt}$</lt>

Where: <lt>$\alpha_t$</lt> is the learning rate at time step <lt>$t$</lt>, <lt>$\alpha_0$</lt> is the initial learning rate, <lt>$k$</lt> is the decay rate, <lt>$t$</lt> is the current time step or iteration.

This was used a function for modifying the learning rate during training and the trianing and validation performance were gathered as in experiment 1.1 across 5 trials. 

The performance of these models trained with the learning rate scheduler was also assessed on the test data, and their performance compared to that of the non-learning rate scheduled models on the test data. 

### Experiment 2 (8 MARKS) <ignore>

#### 2.1 - Dropout Rates

For this experiment, in accordance with the assignment brief, the original training data was re-split into 2 halves to create a new dataset for training. 

A set of dropout rates for experimentation was defined <lt>$0, 0.2, 0.4, 0.6, \text{, and }0.8$</lt>.

Similar to the previous method for training and evaluating a varying learning rate, 5 trials for each dropout rate were carrried out with models initialised with consistent seeding, then trained all other hyperpapramters being fixed and results recorded as above. 

The experiment's results can help show the effect of dropout rgularisation on model performance, and to determine the optimal dropout rate for the DropoutNet model on this specific classification task. 

It allows for the comparison of different dropout rates and their effect on reducing overfitting and improving generalization. The results can guide the selection of an appropriate dropout rate that achieves the best balance between training performance and generalization.

#### 2.2 - Dropout and Transfer Leaning

The second part of this expeiment the experiment aims to investigate the performance of dropout regularization in the context of transfer learning.

It compares the performance of a number of models both in terms of pefrormance during training, but also on the withheld test set. 

The performance of the best performing model from experiment 1 (which did not have any opportunity to have any further training on the new data split) was to be compared with 
*i)* a model trained on the original data without dropout which was been partially retrained on the new data
*ii)* a model trained don the original data *with** dropout which was partially retrained on the new data. 

In both of the latter cases the partial retaining amounted to transfer learning where trained models had some weights 'frozen' (kept fixed) whilst others were reintialised and made trainable on the new data.

Performance for all models was compared during training and validation as well as on the test dataset 

Transfer learning was implimented as follows.

Two models, one with dropout, one without, were initialised and trained as in previous experiments, iterating over 5 random seeds,with  training data for 50 epochs, with performanace monitored across training and validation sets.

The final instance of each model was saved to disk so the trained models weights were stores, along with a record of it;s initial performance on disk. 

The validation and training datasets were then swapped, the models were loaded and all of their layers were frozen except their fully connected layers which were manually re-initialised, meaning they were subject to training. 

These two models were then trained as in previous experiment - 5 times each by iterating over the random seeds, but this time the were trained and validated on the swapped data. 

These two models had effectively been trained twice on slightly differently distributed datasets, and so their final layers had been trained on the new data. 

Their performance on this new dataset as well as on the test set was plotted and stored. 

By conducting this experiment, the performance of the models with and without dropout regularization can be compared in a transfer learning scenario. The use of swapped train and validation data allows for evaluating the models' ability to generalize to a different data distribution.

The averaged results and smoothed plots provide insights into how the pretrained models with and without dropout perform when fine-tuned on the swapped data. The test results on the original test dataloader assess the models' performance on unseen data.

The experiment's results can help determine the effectiveness of dropout regularization in transfer learning and whether the pretrained model with dropout outperforms the model without dropout in this specific scenario. It provides valuable information on the models' ability to adapt to new data distributions and generalize well.


### Experiment 3 (8 MARKS) <ignore>

This experiment investigates the gradient flow in three different neural network models: BaselineNet (without regularization), DropoutNet (with dropout regularization), and BatchNormNet (with batch normalization). The goal was to analyze and compare the mean and standard deviation of the gradients in the first 5 epochs and the last 5 epochs of training for each model.

This was done by extracting the raw gradient values for each layer from the model during training for the first 5 training steps, and the last 5 training steps. 

Conveniently pytorch has this values acessable as aaccessable value in the model. 

# 3.1
this process was done for the baseline model

# 3.2
this process was done for the dropout model

# 3.3
for this experiment it was required to impliment batch normalisation
this process was done for the batch norm model

```
The experiment follows these steps:

1. Set the number of epochs to 30 and the learning rate to 0.05.

2. For the BaselineNet model:
   - Initialize the model and set the random seed to 1984 for reproducibility.
   - Define the loss function (CrossEntropyLoss) and optimizer (SGD) with the specified learning rate.
   - Collect the absolute gradients for the first 5 epochs and the last 5 epochs using the `collect_gradients_abs` function.
   - Compute the mean and standard deviation of the absolute gradients for the first 5 epochs and the last 5 epochs using the `compute_gradient_statistics_abs` function.
   - Plot the mean and standard deviation of the absolute gradients for the first 5 epochs and the last 5 epochs using the `plot_gradient_statistics_abs` function.

3. Repeat step 2 for the DropoutNet model with a dropout rate of 0.6.

4. Repeat step 2 for the BatchNormNet model.

The `collect_gradients_abs` function collects the absolute gradients for the specified epochs during training. It iterates over the batches in the `train_dataloader` and performs the forward pass, loss computation, and backward pass. The absolute gradients of each layer are collected for the first 5 batches of the first epoch and the last 5 batches of the last epoch.

The `compute_gradient_statistics_abs` function computes the mean and standard deviation of the absolute gradients for each layer based on the collected gradients.

The `plot_gradient_statistics_abs` function visualizes the mean and standard deviation of the absolute gradients for each layer using a bar plot. It creates a figure with two subplots: one for the mean gradients and one for the standard deviations. The x-axis represents the layers, and the y-axis represents the mean or standard deviation values. The bars are grouped by the first 5 epochs and the last 5 epochs for comparison.

By running this experiment, you can observe and compare the gradient flow in the BaselineNet, DropoutNet, and BatchNormNet models. The plots will show the mean and standard deviation of the absolute gradients for each layer in the first 5 epochs and the last 5 epochs. This analysis can provide insights into how the gradients evolve during training and how different regularization techniques (dropout and batch normalization) affect the gradient flow compared to the baseline model.

The results can help understand the impact of regularization on the gradient magnitudes and the stability of the gradient flow throughout the training process.
```
# 3.4 bacth norm on performance

the batch norm model was trained as before with 5 instances averaged and performance assessed on training validation and test in comparison to others




# Results (instructions) - 55 MARKS <ignore>
Use the Results section to summarise your findings from the experiments. For each experiment, use the Markdown/text cell to describe and explain your results, and use the code cell (and additional code cells if necessary) to conduct the experiment and produce figures to show your results.

### Experiment 1 (17 MARKS) <ignore>

As can be seen in Fig 1, initial experminetation established reasonable limits within which to select learning rates for further experimentation. Rates of 0.15 and above lead to unusual, erratic behaviour such as that seen in Fig 1.2 where the learning rate is so high that the model cannot converge to an optimal solution and instead overshoots, leading to loss that prompts a response in the other direction etec etc. A high learning rate and also mean a grater sensitiyvity to initial conditions which perhaps explains the exteme ranges of performanc in the model. On the other hand 1.1 shows the other extreme where the learning rate is so low no learning can occur.

<figure><center><img src="./figs/e1/lrchaos.png" width=700><img src="./figs/e1/tranval_no_learn.png" width=700><figcaption style="max-width: 600px"> Fig 1. Showing behavioural extremes for different learning rates: unstable learning at a learning rate of 0.2, and minimal learning at a learning rate of 0.001 </figcaption></center></figure>

The relative performances of different learning rates can be seen in Fig 2. below and are well cummarised in Fig 3. Looking at Fig 2, Perhaps as would be expected as the learning rates get smaller the generalisation gap between the performance training and validation loss and accuracy is slower to develop, and less extreme - showing that with higher learning rates the model is able to fit to the training data more quickly, but also overfit to it more quickly. The impact of the learning rate can also be seen in the smootheness of the lack of volatility in training loss at lower learning rates, although it should be notes there is only very minotr volatility in plot 2.1 of learning rate of realtively high 0.75.

<figure><center><img src="./figs/e1//lr1.png" width=700><img src="./figs/e1/lr2.png" width=700><img src="./figs/e1/lr3.png" width=700><img src="./figs/e1/lr4.png" width=700><img src="./figs/e1/lr5.png" width=700><figcaption style="max-width: 600px"> Fig 2. Performance plots showing individual and averaged training and validation losses and accuracies for models trained with descending learning rates across 50 epochs of training </figcaption></center>

In terms of performance, the test performances in the table in Fig 4. and the smoothed validation performances in Fig 3 give a good overview of how learning rates affect performance, with lower leardning rates leading to a reduced validtion loss at the end of the 50 epochs owing to slower fitting (and overfitting) to the training data. This was mirrored by a much quicker rise to high accurcy for those with high learning rates, but a plateaing and grdual decline after. The lower learning rates were slower to ahcieve high accuracy, but were seemingly also able to arrive at a more fine grained point in paramater space as they ultimate had better performance

<figure><center><img src="./figs/e1/smoothed loss accuracy.png" width=700><figcaption style="max-width: 600px"> Fig 3. Smoothed averaged results for accuracies and losses across 50 epochs on validation data for models trained with different learning rates</figcaption></center></figure>
<figure><center><img src="./figs/e1/leraning rates test performance.PNG" width=400><figcaption style="max-width: 600px"> Fig 4. Test set performance of models trained with different learning rates highlighting the best result for each metric in green</figcaption></center></figure>

1.2.

Having observed the different performances above, it was clear the ideal balance would be a learning rate that began at the highest end of the learning rates above (0.1), but that decayed reasonably quickly in order to avoid the quite quick onset of overiftting around 10 epochs, and then moved down thr learning rates that were still increasing at different accuracies. 


<figure><center><img src="./figs/e1/lr_scheculer experiments.png" width=350><figcaption style="max-width: 600px"> Fig 5. Different learning rate decay schedules affect on the active learning rate across 50 epochs </figcaption></center></figure>

Different approaches to decay and how they affect lr over the 50 epochs can be seen in Fig 5. The smooth inverse time fuinction with a decay rate of 0.25 seemed to have the ideal combination and was found to perform well relative to the others. 

As can be seen when it was used in comparison to a non learning rate scheduled best performing moder from task 1.1 (see fig 1.3), there was is slight improvment in overall performance,  however the most significant different appears to be a stability in the validation performance but it's validation accuracy stabalises in a way that did not occur with the other models that saturated at close to 100% training accuracy before. This is because the even decreasing learning rate means that after a certain point the paramaters will settle as they will only be getting the tiniest updates. 

It is also noteable that this model achieves 100% on the training set, which none of the earlier models did - again owing to (in the case of high learning rate models) being to coarse to home in on a particualr point in paramater space that would give that 100% accuracy, or (in the case of the very low learning rates) possibly being unable to tranverse the landscape owing to too small a gradient, possibly getting stuck in sub-optimal minima.  

<figure><center><img src="./figs/e1/LR SCHEDULER final results.png" width=700><figcaption style="max-width: 600px"> Fig 6. Performance over 50 epochs of training for model trained with learning rate scheduler </figcaption></center></figure>

<figure><center><img src="./figs/e1/results accuracy camparison lr and scheduler.png" width=350><figcaption style="max-width: 600px"> Fig 7. Comparison of performance across training of model trained with a learning rate scheduler, and the best performing model without a scheduler (lr of 0.05)</figcaption></center></figure>
<figure><center><img src="./figs/e1/lr decay comparison.PNG" width=400><figcaption style="max-width: 600px"> Fig 8. Comparison of test results between a model trained with a learning rate scheduler, and the best performing model trained without a scheduler (lr of 0.05) highlighting the best result for each metric in green</figcaption></center></figure>

Overall this experiment demonstrates well the impact that different learning rates can have on learning in a NN model. 

### Experiment 2 (19 MARKS) <ignore>

2.1

The effect of increasing dropout rates can clerly be seen in figs 9.1 to 9.5. All models were initialised with the same paramaters other than the dopout rate and what can be observed demonstrates the effect of dropout as a regularisation technique. As the dorpout rate increases from 0 to 0.8 we see a reduction in the speed and extent to which the model fits to the training data. This is reflected in the final accuracy it obtains on the training data and the speed with which is gets there. It can also be seen in the dingficantl decrease in the the generalisation gap between training and validation performance, where in the absence of dropout (9.1) there is the biggest gap. and the highest dropout leads to the smallest gap.

 <figure><center><img src="./figs/e2/dr0.png" width=800><img src="./figs/e2/dr02.png" width=800><img src="./figs/e2/dr04.png" width=800><img src="./figs/e2/dr046.png" width=800><img src="./figs/e2/dr08.png" width=800><figcaption style="max-width: 600px"> Fig 9. Performance plots showing individual and averaged training and validation losses and accuracies for models trained with increasing dropout rates across 50 epochs of training </figcaption></center></figure>

In Fig10 the comparative performance can be seen clearly, where in the Validation loss, one can the the onset and magnitude of the validation loss developing is earlier and more extreme for the lower dropout rates. It seems that this doesnt have such a devastating effect on the validation accuracy as the lower dropout rates still rech reasonable accuyracies o both test and validation, however the best loss and test performance belongs to those models with higher dropout rates - although it is not an enormous difference, as it is likely to be a true reflection of performance. 

<figure><center><img src="./figs/e2/overall dropout comparisons.png" width=800><figcaption style="max-width: 600px"> Fig 10. Smoothed averaged results for accuracies and losses across 50 epochs on validation data for models trained with different dropout rates </figcaption></center></figure>
<figure><center><img src="./figs/e2/dropout rates test results.PNG" width=300><figcaption style="max-width: 600px"> Fig 11. Test set performance of models trained with different dropout rates highlighting the best result for each metric in green</figcaption></center></figure>



2.2
MODEL NUMBERS: 0 = BASELINE , 1=DROPOUT  
pretrained 

The findings of the above are reiterated in the freshly trained models shown in figure 13 where we see the model trained without dropout (model 0 in these experiments) demonstrating poor generalisability, and marked over fitting. While The model trained with dropout fits less closely to the training data as seen in its poorer performance on the test accuracy, what it does learn is mostly generalised to the validation dataset. The properyies are visable in the comparison plot in Fig 13 as well. 

<figure><center><img src="./figs/e2/pretrained_0.png" width=800><figcaption style="max-width: 600px"> Fig 12. Performance plots showing individual and averaged training and validation losses and accuracies for Baseline (non-dropout) model (model 0)during trained on the original data over 5figcaptionstyle=figcaption></center></figure>

<figure><center><img src="./figs/e2/pretrained_1.png" width=800><figcaption style="max-width: 600px"> Fig 13. Performance plots showing individual and averaged training and validation losses and accuracies for model with dropout implimented (model 1) during trained on the original data over 50 epochs. </figcaption></center></figure>

<figure><center><img src="./figs/e2/pretrained comparison.png" width=400><figcaption style="max-width: 600px"> Fig 14. Direct comparison of performance of averaged and smoothed performance of non-dropout (model 0) and dropout (model 1) models over 50 epochs of training and validation on the original data</figcaption></center></figure>

<figure><center><img src="./figs/e2/pretrained test results.PNG" width=300><figcaption style="max-width: 600px"> Fig 15. Test set performance of models trained without (0) and with (1) dropout implimented highlighting the best result for each metric in green. </figcaption></center></figure>

Both of the models above has their fully connected layers retrained on a reversed version of the original dataset (and so essentially a 'new' dataset in terms of a new distribution) whilst other models remained frozen. Their performance during training can be seen in Fig 16 and 17 where 16 is the dropout-free model and 17 is the dropout model.  

It is clear that the dropout free model fits and then overfits to this new data extremely quickly, with only a very brief period during which it is learning generalisable information from the new data, even more so than trained on just the original data.  

The model trained and retrained with dropout on the other hand still overfits but the process is smoother and more gradual with a less sever transition between fitting with generalisation to then overfitting. 



<figure><center><img src="./figs/e2/retrained_baseline.png" width=800><figcaption width=500> Fig 16. Performance plots showing individual and averaged training and validation losses and accuracies for Baseline (non-dropout) model (model 0) during retraining on swapped data over 50 epochs</figcaption></center></figure>

<figure><center><img src="./figs/e2/retrained_dropout.png" width=800><figcaption style="max-width: 600px">Fig 17. Performance plots showing individual and averaged training and validation losses and accuracies for model with dropout (model 0) during retraining on swapped data over 50 epochs</figcaption></center></figure>

<figure><center><img src="./figs/e2/retrained_comparison.png" width=400><figcaption style="max-width: 600px">Fig 18. Direct comparison of performance of averaged and smoothed performance of non-dropout (model 0) and dropout (model 1) models over 50 epochs of training and validation on the swapped data </figcaption></center></figure>

In terms of overall performance on the test set, the model with dropout performas better on both metrics, although not enormously better. It also performs better than any other model so far other than that which used the learning rate scheduler. 

It can therefor be said that the regularisation effect of a single dropout layer was able to improve performance almost to the same level that basic learning rate scheduling was. 

<figure><center><img src="./figs/e2/retrained comparison test results.PNG" width=300><figcaption style="max-width: 600px"> Fig 19. Test set performance of models retrained on swapped dataset having been previously trained on original dataset without (0) and with (1) dropout implimented highlighting the best result for each metric in green.</figcaption></center></figure>





### Experiment 3 (19 MARKS) <ignore>

3.1

Figs. 20, 21  show various statistics relating to gradient flow through the different models that have been tested so far. Fig 22 shows the same measurement for a new version of the model with Batch Normalisation implimented, and 23 shows the values for the extra layers present in that batch normalised model.

The absolute value of the gradient was used for all statistics as it provides a clearer and more meaningful representation of gradient magnitudes. Absolute values convey the strength of gradients, regardless of whether that gradient is positive or negative, which is more relevant for understanding the propagation of those gradients through the layers. In contrast, averages of non-absolute values can be influenced by the direction of gradients, potentially obscuring the overall magnitude and making it harder to identify issues like vanishing or exploding gradients. Plotting the standard deviation of the absolute gradient values shows the variability in gradient magnitudes, providing insights into the consistency and stability of gradient flow across different layers and iterations. 

In Fig 20 below we can see firstly that for the baseline model gradient in the first 5 episodes the gradient is small, but is larger later layers (closer to the loss on the computational graph) compared to the earlier layers. The gradients are overall higher, but also seem to increase as the layers go back further in the last 5 episodes. Variability seems to be in proportion to the size of the gradient. 

These results indicate that initially there are very small updates being make to primarilty paramaters in the later layers, with little learning occuring in the earlieest layers. By the end of training this has changes significantly and there is more information being passed to the earlier layers.
<figure><center><img src="./figs/e3/gradients baseline model.png" width=800><figcaption style="max-width: 600px"> Fig 20. Mean and standard deviation of the gradients of the loss function with respect to the paramaters at each layer of thebaseline model during training. Values are the average of the gradient of teh paramaters for each layer across the first 5 training episodes and the last 5 episodes of 50 epochs of training. </figcaption></center></figure>

3.2
Fig 21 shows some remarkable similarities to Fig 20. indicating smilarieites in gradient flow between the models with and without dropout. The most significant difference is the magnitude of the gradients, which are higher in both the first and last 5 episodes, though with similar variablity and a very similar pattern of propagation as descibed above. 
<figure><center><img src="./figs/e3/gradients dropout model.png" width=800><figcaption style="max-width: 600px"> Fig 21. Mean and standard deviation of the gradients of the loss function with respect to the paramaters at each layer of the model with dropout implimented during training. Values are the average of the gradient of teh paramaters for each layer across the first 5 training episodes and the last 5 episodes of 50 epochs of training. </figcaption></center></figure>

3.3
Figs 22 and 23 showing the gradient propogation in a batch normalised model are significantly different, as one might expect. Firstly it should be pointed out that all of the bias terms for any layers with batch norm applied dissapear, as the bias effect is essentially taken over by the parameters of the batch norm layer (visable in Fig 23.) due to the 'absorbtion of bias' phenomenon of[x, y].

In the layers that are common, however a number of things are stiking. Firstly the values of the gadients are dramatically higher for all layers in the first 5 episodes which is especially significant for the earlier layers where virtually no gradient was reaching in the un batch normalised models. In the last 5 episodes it is broadly similar. 

The distribution of the gradient is also more consisten with the batch normalised model. Whereas with non-batch normalised models it very much shifts from being mostly updating later layers to then earlier layers, batch norm  is more evenly distributied (with more to the earlier layers) throughout. 

<figure><center><img src="./figs/e3/gradients batchnorm model (matching others).png" width=800><figcaption style="max-width: 600px"> Fig 22. Mean and standard deviation of the gradients of the loss function with respect to the paramaters at each layer of the model with dropout implimented during training. Values are the average of the gradient of teh paramaters for each layer across the first 5 training episodes and the last 5 episodes of 50 epochs of training. Not in this plot the batch normalisation layers and their paramaetyrr gradients are not represented to facilitate comparison with previous models  </figcaption></center></figure>
<figure><center><img src="./figs/e3/gradients batchnorm model (not matching others).png" width=1000><figcaption style="max-width: 600px"> Fig 23. Mean and standard deviation of the gradients of the loss function with respect to the paramaters at each layer of the model with dropout implimented during training. Values are the average of the gradient of teh paramaters for each layer across the first 5 training episodes and the last 5 episodes of 50 epochs of training. batch norm paramater gradients included. </figcaption></center></figure>

<figure><center><img src="./figs/e3/gradint flow relative metrics.png" width=800><figcaption style="max-width: 600px"> Fig 24. Comparison grouped by metric. </figcaption></center></figure>
3.4 

The actual performance of the batch norm model can be seen below in Fig 25 and on the test set in fig 26. It can be seen that the model overfits quickly and performs quite poorly on the test data, with quite a substantial instability in the validation beformance. This is perhaps a surprising result given the regularistion effect batch norm is often associated with [x, y] and shall be sicussed more in the analysis below, as there are other properties of batch norm and the dataset in general which max be more important here. 

<figure><center><img src="./figs/e3/batch norm performance.png" width=800><figcaption style="max-width: 600px"> Fig 25. Performance plots showing individual and averaged training and validation losses and accuracies for a model with batch normalisation applied and trained on orignal data over 50 epochs. </figcaption></center></figure>
<figure><center><img src="./figs/e3/batch norm test results.PNG" width=300><figcaption style="max-width: 600px"> Fig 26. Test performance of model trained with batch normalisation</figcaption></center></figure>



# Conclusions and Discussion (instructions) - 25 MARKS <ignore>
In this section, you are expected to:
* briefly summarise and describe the conclusions from your experiments (8 MARKS).
* discuss whether or not your results are expected, providing scientific reasons (8 MARKS).
* discuss two or more alternative/additional methods that may enhance your model, with scientific reasons (4 MARKS). 
* Reference two or more relevant academic publications that support your discussion. (4 MARKS)

These experiments demonstrated some of the fundamental properties of aritficial nueral networks.

Experiment one demonstrated the effect that the learning can have on a models ability to fit to training data, and the impact that this has on generalisation. It showed that too high a learning rate could lead to coarse updates that lead to instability and variability in performance and and in ability to get to the true optimal minimial loss. It also showed that low learning rates lead to slow progress but more close fit to training data.

It was shown that a learning rate scheduler can balance these properties and lead to quick learning with more fine grained accuracy in later stages of training. However, this did not translate to significant benefits in validation and test performance, although there was some.

Experiment two demonstrated the regularisation effect of dropout both in regular trianing and in a trasnfer learning scenario. It was shown to have a signficant impact on the generalisation gap reducing it as the rate increased and reducing the validation loss signifcantly. It was also found to have a profound regularising effect in the transfer learning example. Although it did lead to improvements in performance this were fairly small. 

Experiment 3 again demonstrated clearly the powerful impact of batch normalisation on the propogation of gradient backwards through they layers of a neural network. The stark contract in average gradients arriving in the early layers in the intial impacts was striking. The impact it had on model performance however was perhaps a bit dissapointing, but I believe can be understood.

The results in experiments one and two are very much to be expected.

Learning rates are known to have a significant impact on the training dynamics and convergence of neural networks. High learning rates can lead to overshooting the optimal solution and oscillations around the minimum, while low learning rates result in slow convergence but more stable updates. The use of learning rate schedulers, such as reducing the learning rate over time, allows for faster initial convergence while fine-tuning the model in later stages. This is consistent with the observations in experiment one. 

The regularization effect of dropout is also well-established. Dropout introduces noise and stochasticity into the network by randomly dropping activation, preventing over-reliance on individual neurons and promoting more robust representations. This leads to improved generalization and reduced overfitting, as demonstrated in experiment two. Experiment 2 also demonstrates the impact that this can have on a networks ability to fit to data - with a reduced performance on the training data going along with the increased accuracy. 

In the transfer learning context, it would be expected that dropout would have a similar regularizing effect, helping the model adapt to the new task while preventing overfitting to the limited new data. The smaller impact on performance in the transfer learning scenario could be attributed to the already well-initialized weights from pretraining, which provide a good starting point for fine-tuning.

There are a number of approaches I could take to enhance the model here; skip connections, a more advanced optimizer such as Adam, but also simply experimenting and tuning my hyperparamters to the new arhictetures that were being added in at differnt stages, as these changes should go hand in hand with other changes.

There are a number of approaches I could take to enhance the model here; skip connections, a more advanced optimizer such as Adam, but also simply experimenting and tuning my hyperparameters to the new architectures that were being added in at different stages, as these changes should go hand in hand with other changes. Early stopping could prove beneficial as a regularization technique to prevent overfitting. By monitoring validation loss during training and terminating when it stops improving, I may be able to enhance generalization performance, albeit with potentially modest gains given other regularizers in place.

Skip connections, employed in architectures like ResNets, facilitate better gradient flow across layers during backpropagation. This can mitigate the vanishing/exploding gradient issue in very deep networks. I'd expect adding skip connections to lead to improved training stability, faster convergence, and better validation performance due to the increased optimization capability.

Using the Adam optimizer instead of basic SGD is another promising avenue. Adam computes adaptive per-parameter learning rates from estimates of the gradients' first and second moments. This often allows for faster convergence to a lower training loss compared to a fixed learning rate. Importantly, these training benefits with Adam have been shown to translate to better generalization as well.

Of course, judicious tuning of hyperparameters like the batch normalization momentum and dropout rates would also be crucial, especially when incorporating architectural changes. These parameters may need recalibration to suit the modified model components and capacity. Experimentation and validation on tuning ranges would go hand-in-hand with testing the skip connections and Adam optimization approaches. A multi-pronged enhancement strategy, combining complementary techniques like these, is likely to yield the most robust performance gains.